In [ ]:
# Author: Alexander Nielson
# Date: 2020-09-22

# this gives an example query and results using the OSA's transaction.transaction Big Query database.
# This example examines Higher Ed (ie: University) fund level revenue, expenditure, and payroll. 

In [2]:
# Libs
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
import numpy as np
import datetime
import re
import requests
import json
import matplotlib.pyplot as plt
import time

You may need to do some googling to get the right modules installed. For example, you might have to run

>pip install google-cloud-bigquery

to get bigquery imported. This is because google changed their client python library to be separate from one giant package. 

In [3]:
pd.set_option('display.float_format', lambda x: '%.2f' % x) # I like my number formatting rounded to 2 decimals.

In [4]:
#Authenticate
key_path = "D:/gcp_keys/ut-sao-transparency-prod-5a8476ec5fea.json" # DEV DO: path to you service account key

credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

BQ = bigquery.Client(
    credentials=credentials,
    project=credentials.project_id,
)

In [5]:
# lets run a query to get all the entity's and their governmental levels. 
q = """
  SELECT DISTINCT entity_name, govt_lvl
  FROM `ut-sao-transparency-prod.transaction.transaction`
    """

In [6]:
entities = BQ.query(q).result() .to_dataframe()

C:\Users\alexnielson\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: PyarrowMissingWarning: Converting to a dataframe without pyarrow installed is often slower and will become unsupported in the future. Please install the pyarrow package.
  """Entry point for launching an IPython kernel.


In [7]:
print(entities.to_string())

                                             entity_name                govt_lvl
0                                 Morgan School District           K12 EDUCATION
1                               Stansbury Service Agency    DISTRICTS AND OTHERS
2                                          Morgan County                  COUNTY
3                                                  Enoch                    CITY
4                                        Dutch John Town                    CITY
5                                       Box Elder County                  COUNTY
6                         American Academy of Innovation           K12 EDUCATION
7                                     North Star Academy           K12 EDUCATION
8                                  Davis School District           K12 EDUCATION
9                                             Wellsville                    CITY
10          Southeastern Utah Association of Governments              INTERLOCAL
11                          

Lets filter to just the Higher Education government entities, since that is what what we are interested in.

In [8]:
q = """
    SELECT entity_name, fiscal_year, fund1, org1, org2, org3, type, cat1, cat2, cat3, description,
    amount, program1, govt_lvl
    FROM `ut-sao-transparency-prod.transaction.transaction`
    WHERE entity_name = "Utah Valley University" and fiscal_year = 2019
"""

In [9]:
parker_test = BQ.query(q).result() .to_dataframe()

C:\Users\alexnielson\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: PyarrowMissingWarning: Converting to a dataframe without pyarrow installed is often slower and will become unsupported in the future. Please install the pyarrow package.
  """Entry point for launching an IPython kernel.


In [10]:
higher_ed = entities[entities['govt_lvl'] == "HIGHER EDUCATION"]

In [11]:
higher_ed

,entity_name,govt_lvl
86,Utah State University,HIGHER EDUCATION
192,Southern Utah University,HIGHER EDUCATION
214,University of Utah,HIGHER EDUCATION
278,Dixie State University,HIGHER EDUCATION
291,UHEAA - Student Loan Guarantee Program,HIGHER EDUCATION
292,Southwest Technical College,HIGHER EDUCATION
322,Ogden-Weber Technical College,HIGHER EDUCATION
324,Uintah Basin Technical College,HIGHER EDUCATION
351,Salt Lake Community College,HIGHER EDUCATION
436,Utah State Board of Regents,HIGHER EDUCATION


Great. Now that we know the entity_name for the universities we are interested in for our analysis, we can filter and write queries using their exact name(s).

In [12]:
# Below is a query that I wrote for Parker to help check for a variety of Higher Ed 
# spendings in regards to Employee Compensation.
q = """
SELECT
  entity_name,
  fiscal_year,
  batch_id,
  type,
  cat1,
  cat2,
  cat3,
  cat4, 
  SUM(amount) AS net_amount
FROM
  `transaction.transaction`
WHERE
  entity_name IN ('Dixie State University',
    'Salt Lake Community College',
    'Snow College',
    'Southern Utah University',
    'University of Utah',
    'Utah State University',
    'Utah Valley University',
    'Weber State University' ) 
    AND type IN ('EX', 'PY') 
GROUP BY 
  entity_name, 
  fiscal_year,
  cat1,
  cat2,
  cat3,
  cat4,
  batch_id,
  type
"""

In [13]:
cat_level_query = BQ.query(q).result() .to_dataframe()

C:\Users\alexnielson\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: PyarrowMissingWarning: Converting to a dataframe without pyarrow installed is often slower and will become unsupported in the future. Please install the pyarrow package.
  """Entry point for launching an IPython kernel.


In [14]:
cat_level_query

,entity_name,fiscal_year,batch_id,type,cat1,cat2,cat3,cat4,net_amount
0,Snow College,2021,B-0003751,EX,Current Exp,Dues,None,None,19369.17
1,Snow College,2021,B-0003751,EX,Current Exp,Data Process Maintenance,None,None,12541.67
2,Snow College,2021,B-0003751,EX,Benefits,Prescription Copay,None,None,-6693.81
3,Salt Lake Community College,2021,B-0004157,EX,Current Expense,Supplies,None,None,574339.24
4,Southern Utah University,2021,B-0004653,EX,Travel,Travel In-State,None,None,88424.07
...,...,...,...,...,...,...,...,...,...
9876,Weber State University,2016,33889,EX,Current Expense,Data Processing Supply and Serv.,None,None,25253.38
9877,Weber State University,2019,66094,EX,Current Expense,Data Processing Supply and Serv.,None,None,5853.90
9878,Weber State University,2015,28713,EX,Current Expense,Data Processing Supply and Serv.,None,None,-27109.36
9879,Weber State University,2020,60962,EX,Current Expense,Data Processing Supply and Serv.,None,None,16265.20


Lets get the USU data only

In [15]:
usu_df = cat_level_query[cat_level_query['entity_name']=="Utah State University"]

In [16]:
usu_df

,entity_name,fiscal_year,batch_id,type,cat1,cat2,cat3,cat4,net_amount
4340,Utah State University,2017,40195,EX,Staff Benefits,None,None,None,7949021.64
4344,Utah State University,2016,29433,EX,Salaries & Wages,None,None,None,19948289.75
4346,Utah State University,2020,61297,EX,Scholarships & Fellowships,None,None,None,1822664.50
4347,Utah State University,2016,33207,EX,CURRENT EXPENSES,None,None,None,15490310.33
4348,Utah State University,2020,60488,EX,Capital Outlay,None,None,None,2753785.35
...,...,...,...,...,...,...,...,...,...
5192,Utah State University,2015,28305,EX,CURRENT EXPENSES,None,None,None,17626554.53
5193,Utah State University,2019,55680,EX,Scholarships & Fellowships,None,None,None,18207994.81
5194,Utah State University,2019,54941,EX,Travel,None,None,None,1516463.88
5195,Utah State University,2015,30095,EX,Staff Benefits,None,None,None,7172678.53


In [ ]:
usu_wages = usu_df[(usu_df['cat1'].str.contains("Salaries")) & (usu_df['type']=="PY")]

In [ ]:
usu_wages.groupby(['fiscal_year','type','cat1'])['net_amount'].sum()

Now do the same process for Southern Utah University

In [ ]:
suu_df = cat_level_query[cat_level_query['entity_name']=="Southern Utah University"]

In [ ]:
suu_df

In [ ]:
suu_wages = suu_df[(suu_df['type']=="PY")]#(suu_df['cat1'].str.contains("Salaries")) &

In [ ]:
suu_wages.groupby(['fiscal_year','type','cat1'])['net_amount'].sum()